In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
import sys
sys.path.append(str(Path.cwd() / 'util/'))

from qubit_templates import *
from functions import *
from ChipDesign import *

# Read configuration file

In [ ]:
JJtype = "manhattan" # manhattan or dolan

config = load_config( ["config/common_purcellfilter.yaml", f"config/{JJtype}_2D_silicon.yaml"] )
# common_config = load_config( f"config/common.yaml" )
# config = load_config( f"config/{JJtype}_2D_silicon.yaml" )
# config = {**common_config, **config} # common config will be overwritten by value in the right 

SimulationDesign = True # Make inverted design for HFSS simulation (This takes a long time...)
QiskitDesign = True

outname = "ICEPP_mino_2D" + JJtype + "PurcellFilter"
if config["JJ_bandage"]:
    outname += "bd"
if config["DCLine_activate"]:
    outname += "_dcline"
if config["JJ_entangle"]:
    outname += "_entangle"    


# Wafer design

In [ ]:
wafer = device_Wafer(config)

# Place chip designs

In [ ]:
D = sweep_chipdesign( config )
D.center = (0,0)

## Remove corners
Grid_width = config["Grid_sweep_array"][0]["x"]
Grid_height = config["Grid_sweep_array"][0]["y"]
for iref, ref in enumerate(D.references[:]):
    if iref == 0 or iref == len(Grid_width)-1 or iref == len(Grid_width) * (len(Grid_height)-1) or iref == len(Grid_width) * len(Grid_height) - 1:
        D.remove(ref)

wafer.add_ref(D)
qp(D)


# Add EB markers

In [ ]:
EBmarkers = device_EBmarkers(marker_pos = config["EBMarker_pos"], layer = config["EBMarker_layer"])
qp(EBmarkers)
wafer.add_ref(EBmarkers)


# Add grid lines

In [ ]:
grid = device_Grid(config)
wafer.add_ref( grid )

qp(wafer)


In [ ]:
wafer.write_gds('output/' + outname)